In [8]:
!pip install scrapy
!pip install praw

     |████████████████████████████████| 176 kB 5.0 MB/s 
     |████████████████████████████████| 53 kB 2.0 MB/s 


In [5]:
import requests
import requests.auth

client_auth = requests.auth.HTTPBasicAuth('PkK2HAiKXWi4lVmek7mu9g','G-vjkM8dq1NuZjwizkEd3UMLgHfhfg')
post_data = {
      "grant_type": "password", "username": "cysbc1999", "password": "cysbc1999"}
headers = {
      "User-Agent": "ChangeMeClient/0.1 by YourUsername"}
response = requests.post("https://www.reddit.com/api/v1/access_token", auth=client_auth, data=post_data, headers=headers)
print(response.json())

{'access_token': '116991958771-OKFo7l6gm28Iwkvrap8celUBEdddJw', 'token_type': 'bearer', 'expires_in': 3600, 'scope': '*'}


In [10]:
import scrapy
import praw
import datetime
class RedditSpider(scrapy.Spider):
	name = "reddit"
	allowed_domains = ["reddit.com"]
	start_urls = [
		"https://www.reddit.com"
	]

	def parse(self, response):
        #使用client id 和secret 进行登陆
		reddit = praw.Reddit(client_id='PkK2HAiKXWi4lVmek7mu9g', client_secret='G-vjkM8dq1NuZjwizkEd3UMLgHfhfg',
			grant_type='client_credentials', user_agent='mytestscripts/1.0')
		
		"""
		sub = reddit.submission(id='9klf7s')
		#print(sub.title)
		#pprint.pprint(vars(sub))
		"""
		#可以通过 subreddit.stream.submissions()来监控某一个子版块出现的新帖子
		#subreddit = reddit.subreddit('dapps')
		#for sub in subreddit.stream.submissions():
        
        #limit=None来获取所有的贴子，默认为100
        #每次得到的属性类别数量可能不一样
		subs = reddit.subreddit('dapps').new(limit=None)
		for sub in subs:
			item = RedditItem()

			item['html'] = response.body
			#print(item['html'])

            #permalink是网站下该帖子的前缀，需要和网站地址拼接构成该帖子的链接地址
			url = 'https://{}{}'.format(self.allowed_domains[0], sub.permalink)
			item['url'] = url
			redditor = sub.author
            #作者可能为空
			#print("author:", redditor.name)
			if redditor is not None:
				item['author'] = redditor.name
			else:
				item['author'] = ""

            #sub.created_utc是一个utc时间戳，需要转换成datetime格式
			#print("created utc:", sub.created_utc)
			item['created_time'] = datetime.datetime.utcfromtimestamp(sub.created_utc)

            #如果帖子本身只是一个超链接，那么sub.selftext为空
			item['selftext'] = sub.selftext
			if sub.is_self==False :
				item['selftext'] = sub.url

			yield item


html_insert = '''insert into reddit_dapps_html(html) values('{html}')'''
reddit_insert = '''insert into reddit_dapps(url, url_md5, title, author, 
created_time, selftext, score, num_comments, upvote_ratio)
          values('{url}', '{url_md5}', '{title}', '{author}', 
          '{created_time}', '{selftext}', '{score}', '{num_comments}', '{upvote_ratio}')'''

def process_item(self, item, spider):
  html = item['html']
  if html:
    item['html'] = html.strip().decode(encoding="utf-8")
  ......

  #将时间格式化
  #created_time
  created_time = item['created_time']
  if created_time:
    item['created_time'] = created_time.strftime("%Y-%m-%d %H:%M:%S")

  selftext = item['selftext']
  if selftext:
    item['selftext'] = selftext.replace('\n', '').replace('  ', ' ')

  sqltext1 = self.html_insert.format(
    html = pymysql.escape_string(item['html']))

  #由于score等是数字，需要先转换为字符串格式
  sqltext2 = self.reddit_insert.format(
    url = pymysql.escape_string(item['url']),
    score = pymysql.escape_string(str(item['score'])),
    num_comments = pymysql.escape_string(str(item['num_comments'])),
    upvote_ratio = pymysql.escape_string(str(item['upvote_ratio'])))
  self.cursor.execute(sqltext1)
  self.cursor.execute(sqltext2)

  return item


def open_spider(self, spider):
    # connet database
      # 选择字符集为'utf8mb4'
    self.connect = pymysql.connect(
        host=self.settings.get('MYSQL_HOST'),
        port=self.settings.get('MYSQL_PORT'),
        db=self.settings.get('MYSQL_DBNAME'),
        user=self.settings.get('MYSQL_USER'),
        passwd=self.settings.get('MYSQL_PASSWD'),
        charset='utf8mb4',
        use_unicode=True)